In [16]:
from requests import get
from bs4 import BeautifulSoup
import json
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [66]:
URL = "https://providence.craigslist.org/"
QUERY = "search/apa"

postal_codes = ['02903', '02904', '02905', '02906', '02907', '02908', '02909', '02910', '02911', '02919', ]

def query_url():
    return URL + QUERY

def filter_line(line):
    if not line or line == 'QR Code Link to This Post':
        return False
    else:
        return True

def map_line(line):
    return line.strip('\n').strip(' ')

def get_description(href):
    response_details = get(href)
    detail_soup = BeautifulSoup(response_details.text, 'html.parser')
    body = detail_soup.find(id="postingbody")
    
    return list(filter(filter_line, map(map_line, body.find_all(text=True))))

def extract_repost(duplicate_rows):
    if duplicate_rows:
        return [row.get('data-repost-of') for row in duplicate_rows.find_all(class_="result-row")]
    else:
        return []

def listing(post):
    title = post.find(class_ = "result-title hdrlnk")
    
    return {
        'price': post.find(class_ = "result-price").text, 
        'title': title.text,
        'href': title.get('href'),
        'description': get_description(title.get('href')),
        'duplicates': extract_repost(post.find(class_="duplicate-rows")),
    }


def extract_listings(postal_code):
    response = get(query_url(), params={'bundleDuplicates':1, 'availabilityMode':0, 'sale_date':'all dates', 'search_distance': 0, 'postal': postal_code})
    html_soup = BeautifulSoup(response.text, 'html.parser')
    posts = html_soup.find_all('li', class_= ['result-row', 'result-row duplicate-row'])
    dup_posts = html_soup.find_all('li', class_= 'result-row duplicate-row')

    print("Got type: {}; listings: {}; dup listings: {} for {}".format(type(posts), len(posts), len(dup_posts), postal_code))

    # Is there pagination? Seems to match UI well but not exactly?
    extracted = [listing(post) for post in posts + dup_posts]
   
    with open('craigslist/raw/{}.html'.format(postal_code), 'w') as f:
        f.write(response.text)

    with open('craigslist/extracted/{}.json'.format(postal_code), 'w') as f:
        json.dump(extracted, f, indent=2)

        
extract_listings('02905')
        
# for postal_code in postal_codes:
#     extract_listings(postal_code)
        

Got type: <class 'bs4.element.ResultSet'>; listings: 70; dup listings: 5 for 02905
